In [25]:
import pandas as pd

import pandas as pd
import numpy as np
import os
from google.colab import drive

from mlxtend.plotting import heatmap
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from scipy.stats import randint
from imblearn.over_sampling import RandomOverSampler
import scipy.stats as ss
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder


import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns
import plotly.express as px
from tabulate import tabulate


%matplotlib inline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer, KNNImputer

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
split_path = '/content/drive/MyDrive/WIDS-Datathon-2024/split/'
path = '/content/drive/MyDrive/WIDS-Datathon-2024/Data/Test/'

train_df = pd.read_csv(split_path + 'train_set.csv')
val_df = pd.read_csv(split_path + 'val_set.csv')
test_df = pd.read_csv(path + 'test.csv')
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(10324, 80)
(2582, 80)
(5792, 82)


In [27]:
data_types = train_df.dtypes
categorical_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()

numerical_cols = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

if "patient_zip3" in numerical_cols:
    numerical_cols.remove("patient_zip3")

if "patient_zip3" not in categorical_cols:
    categorical_cols.append("patient_zip3")


numerical_cols.remove('DiagPeriodL90D')
print("Categorical columns:", categorical_cols)
print("Numerical columns:", numerical_cols)



Categorical columns: ['patient_race', 'payer_type', 'patient_state', 'patient_gender', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'Region', 'Division', 'patient_zip3']
Numerical columns: ['patient_id', 'patient_age', 'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19', 'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s', 'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married', 'widowed', 'family_size', 'family_dual_income', 'income_household_median', 'income_household_under_5', 'income_household_5_to_10', 'income_household_10_to_15', 'income_household_15_to_20', 'income_household_20_to_25', 'income_household_25_to_35', 'income_household_35_to_50', 'income_household_50_to_75', 'income_household_75_to_100', 'income_household_100_to_150', 'income_household_150_over', 'income_household_six_figure', 'income_individual_median', 'home_ownership', 'housing_units', 'home_value', 'rent_median'

In [28]:
train_data = train_df.dropna(subset=['patient_state'])
predict_data = train_df[train_df['patient_state'].isnull()]

features = ['Division', 'patient_zip3']
X_train = train_data[features]

print(X_train.head())
y_train = train_data['patient_state']

encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train)

# Extract features for prediction
X_predict = predict_data[features]
X_predict_encoded = encoder.transform(X_predict)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_encoded, y_train)

predicted_states = rf.predict(X_predict_encoded)

train_df.loc[train_df['patient_state'].isnull(), 'patient_state'] = predicted_states


             Division  patient_zip3
0             Pacific           928
1  East North Central           442
2             Pacific           912
3  East South Central           404
4  West North Central           550


In [29]:

X_val = val_df[features]

X_val_encoded = encoder.transform(X_val)

predicted_states_val = rf.predict(X_val_encoded)

val_df['predicted_patient_state'] = predicted_states_val

val_df.loc[val_df['patient_state'].isnull(), 'patient_state'] = predicted_states_val[:len(val_df[val_df['patient_state'].isnull()])]



X_test = test_df[features]

X_test_encoded = encoder.transform(X_test)

predicted_states_test = rf.predict(X_test_encoded)

test_df['predicted_patient_state'] = predicted_states_test

test_df.loc[test_df['patient_state'].isnull(), 'patient_state'] = predicted_states_test[:len(test_df[test_df['patient_state'].isnull()])]



In [30]:
print(train_df['patient_state'].isna().sum())
print(val_df['patient_state'].isna().sum())
print(test_df['patient_state'].isna().sum())

0
0
0


In [31]:
train_df.head()

,patient_id,patient_race,payer_type,patient_state,patient_zip3,patient_age,patient_gender,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,...,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
0,147122,White,NaN,CA,928,63,F,C50919,Malignant neoplasm of unsp site of unspecified...,C7951,...,8.957576,10.109091,8.057576,30.606061,7.018182,4.103030,42.301121,8.487175,20.113179,1
1,541685,NaN,COMMERCIAL,OH,442,60,F,C50511,Malig neoplm of lower-outer quadrant of right ...,C773,...,12.651163,7.497619,0.388095,26.216667,5.111628,7.232558,39.095136,7.912494,14.903156,1
2,139722,White,COMMERCIAL,CA,912,41,F,C50912,Malignant neoplasm of unspecified site of left...,C7801,...,11.420000,12.080000,20.020000,28.266667,6.880000,1.850000,45.598336,9.301945,19.540711,1
3,565716,NaN,COMMERCIAL,KY,404,65,F,1748,Malignant neoplasm of other specified sites of...,C773,...,20.404348,20.813636,0.350000,30.152174,6.473913,5.908696,39.947326,7.622672,9.154618,0
4,948362,NaN,MEDICARE ADVANTAGE,MN,550,67,F,C50412,Malig neoplasm of upper-outer quadrant of left...,C7951,...,11.118841,6.371014,0.649275,29.115942,4.311594,7.723188,35.813548,6.252060,10.602760,1


# Impute payer_type column

In [32]:
# Define age group ranges
age_bins = [0, 18, 30, 40, 50, 60, 70, 80, float('inf')]  # Example age group ranges

age_labels = ['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '71-80', '80+']

# Create the 'age_group' column using pd.cut()
train_df['age_group'] = pd.cut(train_df['patient_age'], bins=age_bins, labels=age_labels, right=False)
train_df['age_group'].isna().sum()
val_df['age_group'] = pd.cut(val_df['patient_age'], bins=age_bins, labels=age_labels, right=False)
test_df['age_group'] = pd.cut(test_df['patient_age'], bins=age_bins, labels=age_labels, right=False)


In [33]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

train_df[numerical_cols] = imputer.fit_transform(train_df[numerical_cols])

val_df[numerical_cols] = imputer.transform(val_df[numerical_cols])

test_df[numerical_cols] = imputer.transform(test_df[numerical_cols])


In [34]:
# Check for NaN values after imputation
print(train_df.isnull().sum())


patient_id           0
patient_race      5108
payer_type        1448
patient_state        0
patient_zip3         0
                  ... 
Ozone                0
PM25                 0
N02                  0
DiagPeriodL90D       0
age_group            0
Length: 81, dtype: int64


In [35]:
categorical_features = ['patient_zip3', 'patient_state', 'age_group']
numerical_features = ['income_household_under_5', 'income_household_5_to_10', 'income_household_10_to_15',
                      'income_household_15_to_20', 'income_household_20_to_25', 'income_household_25_to_35',
                      'income_household_35_to_50', 'income_household_50_to_75', 'income_household_75_to_100',
                      'income_household_100_to_150', 'income_household_150_over', 'income_individual_median',
                      'income_household_median', 'rent_burden']

# Preprocess the features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Prepare the features and target variable
train_data = train_df.dropna(subset=['payer_type'])
predict_data = train_df[train_df['payer_type'].isnull()]

X_train = train_data[categorical_features + numerical_features]
y_train = train_data['payer_type']
X_predict = predict_data[categorical_features + numerical_features]

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_predict_preprocessed = preprocessor.transform(X_predict)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_preprocessed, y_train)

# Predict the missing "payer_type" values
predicted_payer_types = rf.predict(X_predict_preprocessed)

# Impute the missing values in the original dataframe
train_df.loc[train_df['payer_type'].isnull(), 'payer_type'] = predicted_payer_types


In [36]:
X_val = val_df[categorical_features + numerical_features]

X_val_preprocessed = preprocessor.transform(X_val)

predicted_payer_types_val = rf.predict(X_val_preprocessed)

val_df.loc[val_df['payer_type'].isnull(), 'payer_type'] = predicted_payer_types_val[:len(val_df[val_df['payer_type'].isnull()])]


In [38]:
X_test = test_df[categorical_features + numerical_features]

X_test_preprocessed = preprocessor.transform(X_test)
X_test_preprocessed_df = pd.DataFrame(X_test_preprocessed)


# Make predictions on the preprocessed test data
predicted_payer_types_test = rf.predict(X_test_preprocessed)

# Fill in the 'payer_type' column for rows where it was originally missing
test_df.loc[test_df['payer_type'].isnull(), 'payer_type'] = predicted_payer_types_test[:len(test_df[test_df['payer_type'].isnull()])]


In [39]:
print(train_df['patient_state'].isna().sum())
print(val_df['patient_state'].isna().sum())
print(test_df['patient_state'].isna().sum())


print(train_df['payer_type'].isna().sum())
print(train_df['payer_type'].isna().sum())


0
0
0
0
0


# Impute the column "patient_race"


In [40]:
cat_features = ['patient_zip3', 'patient_state']
num_features = ['income_household_under_5', 'income_household_5_to_10', 'income_household_10_to_15',
                      'income_household_15_to_20', 'income_household_20_to_25', 'income_household_25_to_35',
                      'income_household_35_to_50', 'income_household_50_to_75', 'income_household_75_to_100',
                      'income_household_100_to_150', 'income_household_150_over', 'income_individual_median',
                      'income_household_median', 'rent_burden']

# Preprocess the features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ])

# Prepare the features and target variable
train_data = train_df.dropna(subset=['patient_race'])
predict_data = train_df[train_df['patient_race'].isnull()]

X_train = train_data[cat_features + num_features]
y_train = train_data['patient_race']
X_predict = predict_data[cat_features + num_features]

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_predict_preprocessed = preprocessor.transform(X_predict)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_preprocessed, y_train)

predicted_payer_types = rf.predict(X_predict_preprocessed)

# Impute the missing values in the original dataframe
train_df.loc[train_df['patient_race'].isnull(), 'patient_race'] = predicted_payer_types


In [41]:
X_val = val_df[cat_features + num_features]

X_val_preprocessed = preprocessor.transform(X_val)

predicted_payer_types_val = rf.predict(X_val_preprocessed)

val_df.loc[val_df['patient_race'].isnull(), 'patient_race'] = predicted_payer_types_val[:len(val_df[val_df['patient_race'].isnull()])]


In [43]:
X_test = test_df[cat_features + num_features]

X_test_preprocessed = preprocessor.transform(X_test)

predicted_payer_types_test = rf.predict(X_test_preprocessed)

test_df.loc[test_df['patient_race'].isnull(), 'patient_race'] = predicted_payer_types_test[:len(test_df[test_df['patient_race'].isnull()])]


In [44]:
print(train_df['patient_state'].isna().sum())
print(val_df['patient_state'].isna().sum())
print(test_df['patient_state'].isna().sum())


print(train_df['payer_type'].isna().sum())
print(val_df['payer_type'].isna().sum())
print(test_df['payer_type'].isna().sum())


print(train_df['patient_race'].isna().sum())
print(val_df['patient_race'].isna().sum())
print(test_df['payer_type'].isna().sum())


0
0
0
0
0
0
0
0
0


In [45]:
path = '/content/drive/MyDrive/WIDS-Datathon-2024/Hamsini/imputed_values/'
train_df.to_csv(path + 'train_df.csv')
val_df.to_csv(path + 'val_df.csv')
test_df.to_csv(path + 'test_df.csv')